Import libraries

In [ ]:
# Install necessary packages
!pip install pandas scikit-learn tensorflow



For training validation and testing


In [8]:
import csv
import chardet
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math

# Load the dataset
dataset = pd.read_csv('/content/CrossValidation_Dataset.csv', encoding='latin-1')

#Encoding
with open('CrossValidation_Dataset.csv', 'rb') as f:
    result = chardet.detect(f.read(100000))

encoding = result['encoding']
print(encoding)

dataset = pd.read_csv('/content/CrossValidation_Dataset.csv', encoding=encoding)

# Extract features and target variable
X = dataset[['WindSpeed(m/s)', 'WindDirection(°)', 'TheoreticalPowerCurve(KWh)']]
y = dataset['LVActivePower(kW)']

# Normalize features
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

# Split the data into train, validate, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_normalized, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1), kernel_regularizer=l1(0.01)))
model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mae'])

# Reshape data for LSTM (add time dimension)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the test set
loss, mae = model.evaluate(X_test, y_test)
print(f'Test Mean Absolute Error: {mae}')

# Assuming 'y_test_last_20_pred' is a 2D array
# Inverse transform the normalized predictions to get actual ActivePower values
scaler = MinMaxScaler()
y = y.values.reshape(-1, 1)
scaler.fit(y)

# Get last 1440 rows from test set
last_1440 = X_test.shape[0] - 1440

X_test_last_1440 = X_test[last_1440:]
y_test_last_1440 = y_test[last_1440:]

# Make predictions
y_pred_last_1440 = model.predict(X_test_last_1440)

# Inverse transform predictions
y_pred_last_1440 = scaler.inverse_transform(y_pred_last_1440).flatten()

# Calculate RMSE
rmse = math.sqrt(mean_squared_error(y_test_last_1440, y_pred_last_1440))

# Create DataFrame
result_df = pd.DataFrame({
    'Actual': y_test_last_1440,
    'Predicted': y_pred_last_1440,
    'RMSE': rmse
})

print(result_df)

# Create rows to write
rows = [
    [actual, predicted, rmse]
    for actual, predicted in zip(y_test_last_1440, y_pred_last_1440)
]


# Open csv file for writing
with open('CrossValidation_predictions.csv', 'w') as f:

    # Create csv writer
    writer = csv.writer(f)

    # Write column headers
    writer.writerow(['Actual', 'Predicted', 'RMSE'])

    # Write each row
    writer.writerows(rows)

print('CSV saved successfully!')



ISO-8859-1
Epoch 1/50
54/54 [==============================] - 2s 10ms/step - loss: 2555398.5000 - mae: 1081.4441 - val_loss: 1413809.7500 - val_mae: 1132.6912
Epoch 2/50
54/54 [==============================] - 0s 6ms/step - loss: 1015396.5000 - mae: 917.0554 - val_loss: 726065.3125 - val_mae: 759.7629
Epoch 3/50
54/54 [==============================] - 0s 7ms/step - loss: 668671.7500 - mae: 686.6647 - val_loss: 541982.6250 - val_mae: 572.3744
Epoch 4/50
54/54 [==============================] - 0s 7ms/step - loss: 552059.7500 - mae: 551.0490 - val_loss: 495215.7188 - val_mae: 488.2679
Epoch 5/50
54/54 [==============================] - 0s 7ms/step - loss: 525182.9375 - mae: 500.8105 - val_loss: 517746.5312 - val_mae: 487.1635
Epoch 6/50
54/54 [==============================] - 0s 5ms/step - loss: 513655.9062 - mae: 485.1352 - val_loss: 476587.5938 - val_mae: 456.2891
Epoch 7/50
54/54 [==============================] - 0s 5ms/step - loss: 503954.1250 - mae: 471.1853 - val_loss: 471457.

Future Prediction - 24 hrs


In [9]:
import numpy as np

# Train model only on training data
model.fit(X_train, y_train, epochs=50)

# No of features
no_features = X_train.shape[1]

# Create dummy 2D input
next_1440 = pd.DataFrame(np.zeros((1440, no_features)))

# Make predictions
y_pred_next_1440 = model.predict(next_1440)

# Inverse transform predictions
y_pred_next_1440 = scaler.inverse_transform(y_pred_next_1440)

# RMSE will be Nan as we don't have actual values
rmse = [np.nan]*len(y_pred_next_1440)

# Prepare rows for writing to CSV
rows = [
    [np.nan, pred, rmse_val]
    for pred, rmse_val in zip(y_pred_next_1440, rmse)
]

# Write predictions to CSV
with open('CrossValidation_future_predictions.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Predicted'])
    writer.writerows(rows)


print('Future predictions saved to CSV')

Epoch 1/50
54/54 [==============================] - 0s 4ms/step - loss: 382078.8750 - mae: 321.5054
Epoch 2/50
54/54 [==============================] - 0s 4ms/step - loss: 377519.2812 - mae: 321.2458
Epoch 3/50
54/54 [==============================] - 0s 4ms/step - loss: 380061.8125 - mae: 318.3815
Epoch 4/50
54/54 [==============================] - 0s 4ms/step - loss: 378051.7188 - mae: 319.3122
Epoch 5/50
54/54 [==============================] - 0s 4ms/step - loss: 377628.4062 - mae: 312.4259
Epoch 6/50
54/54 [==============================] - 0s 4ms/step - loss: 365353.7500 - mae: 302.7413
Epoch 7/50
54/54 [==============================] - 0s 4ms/step - loss: 368673.9688 - mae: 307.5145
Epoch 8/50
54/54 [==============================] - 0s 4ms/step - loss: 370617.8438 - mae: 309.7080
Epoch 9/50
54/54 [==============================] - 0s 4ms/step - loss: 369076.5312 - mae: 306.5642
Epoch 10/50
54/54 [==============================] - 0s 4ms/step - loss: 369883.3438 - mae: 307.3320

Please use following link to see output files and resources - https://github.com/MIHIR-RANJAN/Wind_Turbine_Power_Prediction_Problem_SE20UARI020_SE20UARI095_SE20UARI112.git